# v2 - TAP Model
- Date: 31/01/2023
- Based on: Zsolts serpent model (in TAPreactor folder)

Questions:
1. S(a,b) for moderator? (Zr, and for H)

In [1]:
FAST = True

# Fuel properties
FUEL_TEMP = 900 # K
# FUEL_ENRICHMENT = 4.0 # wo% # NOT NEEDED IN THIS MODEL
FUEL_R = 5 # cm

# Moderator properties
MODERATOR_TEMP = 900 # K
MODERATOR_R = 1.15 # cm

# Cladding properties
CLADDING_TEMP = 900 # K
CLADDING_R = 1.25 # cm

# Lattice properties
LATTICE_ELEMENTS = 18 # Amount of elements per side
LATTICE_PITCH = 15 # cm

# Containment properties
CONTAINMENT_R = 120 # cm
CONTAINMENT_THICKNESS = 5 # cm -> total 125 cm radius, -> 250 cm diameter

# Core properties
CORE_HEIGHT = 200 # cm

In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
import os

# @@@@@@@@@@@@@@@@@@@@@@ Materials @@@@@@@@@@@@@@@@@@@@@@
# Fuel
material_fuel = openmc.Material(1, "ULiF", temperature=FUEL_TEMP)
material_fuel.add_nuclide('U235', 3.11e-2, 'wo')
material_fuel.add_nuclide('U238', 5.909e-1, 'wo')
material_fuel.add_nuclide('Li7', 4.8358e-2, 'wo')
material_fuel.add_nuclide('Li6', 2.418e-6, 'wo')
material_fuel.add_nuclide('F19', 3.2964e-1, 'wo')
material_fuel.set_density('g/cm3', 5.01)

# Cladding
material_cladding = openmc.Material(2, "ZrH", temperature=CLADDING_TEMP)
material_cladding.add_nuclide('Zr90', 4.9793e-1, 'wo')
material_cladding.add_nuclide('Zr91', 1.0980e-1, 'wo')
material_cladding.add_nuclide('Zr92', 1.6967e-1, 'wo')
material_cladding.add_nuclide('Zr94', 1.7569e-1, 'wo')
material_cladding.add_nuclide('Zr96', 2.8908e-2, 'wo')
material_cladding.add_nuclide('H1', 1.8007e-2, 'wo')
material_cladding.add_nuclide('H2', 4.1389e-6, 'wo')
material_cladding.set_density('g/cm3', 5.66)

# Moderator
material_moderator = openmc.Material(3, "CSi", temperature=MODERATOR_TEMP)
material_moderator.add_nuclide('C12', 2.9936e-1, 'wo')
material_moderator.add_nuclide('Si28', 6.4365e-1, 'wo')
material_moderator.add_nuclide('Si29', 3.3866e-2, 'wo')
material_moderator.add_nuclide('Si30', 2.3120e-2, 'wo')
material_moderator.set_density('g/cm3', 3.21)

# Create Materials list and export to XML
mats = openmc.Materials([material_fuel, material_cladding, material_moderator])
mats.export_to_xml()
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



# @@@@@@@@@@@@@@@@@@@@@@ Geometry @@@@@@@@@@@@@@@@@@@@@@
# # Planes
# plane_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2, boundary_type='vacuum')
# plane_top = openmc.ZPlane(z0=CORE_HEIGHT/2, boundary_type='vacuum')

# # Cylinders
# r_pin = openmc.ZCylinder(r=FUEL_R)
# r_cladding = openmc.ZCylinder(r=CLADDING_R)

# # Regions
# region_height_bound = +plane_bottom & -plane_top
# region_fuel = -r_pin & region_height_bound
# region_cladding = +r_pin & -r_cladding & region_height_bound
# region_moderator = +r_cladding & region_height_bound

# # Cells
# cell_fuel = openmc.Cell(name="fuel", fill=material_fuel, region=region_fuel)
# cell_cladding = openmc.Cell(name="cladding", fill=material_cladding, region=region_cladding)
# cell_moderator = openmc.Cell(name="moderator", fill=material_moderator, region=region_moderator)

# # Universe
# universe_pin = openmc.Universe(cells=(cell_fuel, cell_cladding, cell_moderator))

# # Outer universe
# cell_all_moderator = openmc.Cell(name="outer", fill=material_moderator)
# universe_outer = openmc.Universe(cells=(cell_all_moderator,))
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



# @@@@@@@@@@@@@@@@@@@@@@ Lattice @@@@@@@@@@@@@@@@@@@@@@
# lattice = openmc.HexLattice()
# lattice.center = (0., 0.)
# lattice.pitch = (LATTICE_PITCH,)
# lattice.outer = universe_outer
# lattice.orientation = 'x' # With orientation 'x', the lattice is rotated 90 degrees

# universes = [[universe_pin]*(6*i + (i==0)*1) for i in range(LATTICE_RINGS)[::-1]]

# # Add another outer lattice ring with 4 pin cells on each side of the hexagon
# # This function picks out 4 cells on each side of the hexagon
# def condition_on_ring(i):
#     return 2 < i % LATTICE_RINGS < 7
# if LATTICE_RINGS != 9:
#     raise Exception("LATTICE_RINGS must be 9 - otherwise, change the condition above")
# universes.insert(0, [universe_pin if condition_on_ring(i) else universe_outer for i in range(6*(LATTICE_RINGS))])

# lattice.universes = universes

# plane_containment_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2+CONTAINMENT_THICKNESS, boundary_type='vacuum')
# plane_containment_top = openmc.ZPlane(z0=CORE_HEIGHT/2-CONTAINMENT_THICKNESS, boundary_type='vacuum')
# r_containment_inner = openmc.ZCylinder(r=CONTAINMENT_R)
# r_containment_outer = openmc.ZCylinder(r=CONTAINMENT_R+CONTAINMENT_THICKNESS, boundary_type='vacuum')

# bottom_slab = +plane_bottom & -plane_containment_bottom
# top_slab = -plane_top & +plane_containment_top
# region_height_bound_inner = +plane_containment_bottom & -plane_containment_top
# region_containment = (+r_containment_inner & -r_containment_outer & region_height_bound) | (-r_containment_inner & (bottom_slab | top_slab))

# cell_containment = openmc.Cell(name="containment", fill=material_cladding, region=region_containment)
# cell_main = openmc.Cell(fill=lattice, region=(-r_containment_inner & region_height_bound_inner))
# geometry = openmc.Geometry([cell_containment, cell_main])
# geometry.export_to_xml()
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [ ]:
# @@@@@@@@@@@@@@@@@@@@@@ Plots @@@@@@@@@@@@@@@@@@@@@@

# PLOT_COLOR_BY = 'material'
# PLOT_PIXELS = (1000, 1000)
# PLOT_FUEL_COLOR = 'yellowgreen'
# PLOT_CLADDING_COLOR = 'lightslategray'
# PLOT_MODERATOR_COLOR = 'midnightblue'


# colors = {
#     material_fuel: PLOT_FUEL_COLOR,
#     material_cladding: PLOT_CLADDING_COLOR,
#     material_moderator: PLOT_MODERATOR_COLOR
# }

# plotXY = openmc.Plot.from_geometry(geometry, basis='xy')
# plotXY.color_by = PLOT_COLOR_BY
# plotXY.filename = 'plotXY'
# plotXY.pixels = PLOT_PIXELS
# plotXY.colors = colors
# plotXY.to_ipython_image()

# plotXZ = openmc.Plot.from_geometry(geometry, basis='xz')
# plotXZ.color_by = PLOT_COLOR_BY
# plotXZ.filename = 'plotXZ'
# plotXZ.pixels = PLOT_PIXELS
# plotXZ.colors = colors
# plotXZ.to_ipython_image()


# # plots = openmc.Plots([plot])
# # plots.export_to_xml()
# # set xlabel of plot

# # vox_plot = openmc.Plot()
# # vox_plot.type = 'voxel'
# # vox_plot.width = (100., 100., 50.)
# # vox_plot.pixels = (400, 400, 200)
# # vox_plot.to_ipython_image()
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
